In [13]:
import pandas as pd
import numpy as np
#from google.colab import drive
from statsmodels.tools.eval_measures import rmse

In [2]:
#drive.mount('/content/drive')

In [3]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [4]:
filtro_electronics = pd.read_csv('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Electronics_train_v3.csv?raw=true')

In [5]:
filtro_electronics.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter
0,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,17-3-2018,15,2018-03-17,17,3,2018,keyboard great condit work like came box port ...,razer blackwidow chroma keyboard,razer
1,Galaxy S7 Edge (Unlocked) 32GB,3,Electronics/Cell Phones & Accessories/Cell Pho...,Samsung,386.0,0,"Reasonable offers welcomed. But if you ask ""lo...",15-9-2018,6,2018-09-15,15,9,2018,reason offer welcom ask lowest lowbal ill bloc...,galaxi s7 edg ( unlock ) 32gb,samsung
2,Triple car charger,1,Electronics/Cell Phones & Accessories/Chargers...,Other,8.0,1,Brand new never used All colors are available ...,20-6-2018,48,2018-06-20,20,6,2018,brand new never use color avail rm,tripl car charger,basics_brand
3,Otterbox Defender iPhone 6 Plus/6s Plus,1,"Electronics/Cell Phones & Accessories/Cases, C...",Apple,13.0,1,Brand new Otterbox Defender iPhone 6 Plus/6s Plus,18-2-2018,2,2018-02-18,18,2,2018,brand new otterbox defend iphon 6 plus6 plu,otterbox defend iphon 6 plus/6 plu,apple
4,XBOX One,3,Electronics/Video Games & Consoles/Consoles,Xbox,226.0,0,500GB Comes with power cord One controller wit...,10-4-2018,26,2018-10-04,4,10,2018,500gb come power cord one control recharg pack...,xbox one,xbox


In [6]:
filtro_electronics['sub_category_name'] = sub_categoria(filtro_electronics)

In [7]:
filtro_electronics.sub_category_name.value_counts()

Cell Phones & Accessories    25075
Video Games & Consoles       17599
Media                         7063
TV, Audio & Surveillance      6862
Computers & Tablets           5805
Cameras & Photography         3092
Other                          593
Car Audio, Video & GPS         425
Name: sub_category_name, dtype: int64

# modelo 01

Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name e brands_filter

In [8]:
import statsmodels.api as sm

In [9]:
filtro_electronics = pd.get_dummies(filtro_electronics, columns=['sub_category_name','brands_filter'])

In [10]:
X = filtro_electronics.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_electronics.price
X = sm.add_constant(X)
m1_rl = sm.OLS(exog=X,endog=y)
m1_result = m1_rl.fit()
print(m1_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     490.7
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        15:25:11   Log-Likelihood:            -3.7326e+05
No. Observations:               66514   AIC:                         7.466e+05
Df Residuals:                   66468   BIC:                         7.470e+05
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [11]:
filtro_electronics['price'].describe()

count    66514.000000
mean        45.112563
std         76.414868
min          3.000000
25%         10.000000
50%         20.000000
75%         44.000000
max       1909.000000
Name: price, dtype: float64

In [14]:
m1_ypred = m1_result.predict(X)
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

66.2

# Modelo 02
Mantem caracteristicas do modelo 1 com filtro de preço ate $100,00

In [15]:
filtro_electronics_ajustado = filtro_electronics[filtro_electronics['price'] <= 100]

In [16]:
X1 = filtro_electronics_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_electronics_ajustado.price
X1 = sm.add_constant(X1)
m2_rl = sm.OLS(exog=X1,endog=y1)
m2_result = m2_rl.fit()
print(m2_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     1163.
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        15:26:43   Log-Likelihood:            -2.4714e+05
No. Observations:               59490   AIC:                         4.944e+05
Df Residuals:                   59445   BIC:                         4.948e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [17]:
filtro_electronics_ajustado['price'].describe()

count    59490.000000
mean        24.871054
std         21.030295
min          3.000000
25%         10.000000
50%         17.000000
75%         34.000000
max        100.000000
Name: price, dtype: float64

In [18]:
m2_ypred = m2_result.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

15.42

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [20]:
X2 = filtro_electronics.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_electronics.price

In [21]:
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)

In [22]:
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5

In [23]:
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")

R² = 0.33
MSE = 3902.08
RMSE = 62.47


# Modelo 04

Arvore de Regressão usando como criterio de seleção dos nós a métrica R² e com filtro de preço ate $100,00

In [24]:
X3 = filtro_electronics_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_electronics_ajustado.price

In [25]:
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)

In [26]:
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.53
MSE = 208.82
RMSE = 14.45


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 24,9% | 66,20 | Regressão Linear |
| Modelo 02 | 46,3% | 15,42 | Regressão Linear (preço <= 100) |
| Modelo 03 | 33,0% | 62,47 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 53,0% | 14,45 | Arvore Regressão (criterio = R², preço <= 100) |